![Callysto.ca Banner](https://github.com/callysto/curriculum-notebooks/blob/master/callysto-notebook-banner-top.jpg?raw=true)

<a href="https://hub.callysto.ca/jupyter/hub/user-redirect/git-pull?repo=https%3A%2F%2Fgithub.com%2Fcallysto%2Fdata-science-and-artificial-intelligence&branch=main&subPath=06-data-analysis-spotify.ipynb&depth=1" target="_parent"><img src="https://raw.githubusercontent.com/callysto/curriculum-notebooks/master/open-in-callysto-button.svg?sanitize=true" width="123" height="24" alt="Open in Callysto"/></a>

# Data Analysis with Spotify Data

[Spotify](https://en.wikipedia.org/wiki/Spotify), an audio streaming platform, has a huge database of songs and information about them. *If you'd like to create your own data sets, you can check out the [Getting Spotify Data notebook](06b-getting-spotify-data.ipynb).*



`Run` the cell below to import a dataset of about 40,000 songs that has been [exported from Spotify](https://developer.spotify.com/documentation/web-api).

In [1]:
import pandas as pd
data = pd.read_csv('https://raw.githubusercontent.com/callysto/data-files/main/data-science-and-artificial-intelligence/spotify.csv')
data

,track,artist,track_id,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,chorus_hit,sections,popularity,release_date
0,Country Livin',The Lacs,1GgJ8J0Xuq3yKhirm6TIeP,0.692,0.902,5,-5.013,1,0.1590,0.155000,0.000000,0.1440,0.724,156.880,239851,4,59.70002,7,0,2018-05-04
1,Hopeful Shivering,Apocryphal,1nTrGqFfOubot9Vh6rt6cb,0.547,0.647,11,-9.015,0,0.0371,0.124000,0.007520,0.1040,0.182,120.045,392375,4,50.51804,17,0,2016-10-15
2,Punk Rawk Christmas,MxPx,1aWd5MkGpyhqENRiilEWxe,0.486,0.969,9,-4.457,1,0.0739,0.010800,0.000000,0.0876,0.827,148.067,200573,4,26.46958,10,0,2018-11-30
3,I Smile,Kirk Franklin,0UQDSP8cz6WmLJckO5jqUQ,0.745,0.753,1,-2.408,1,0.1640,0.223000,0.000000,0.7050,0.589,144.057,307973,4,87.93854,9,1,2011-03-21
4,"Never Created, Never Destroyed",Jlin,6tQTZJ3lvQiJLVpKDJ47Vp,0.870,0.386,7,-8.081,1,0.4640,0.000628,0.225000,0.0322,0.734,157.845,211711,3,34.71437,10,0,2017-05-19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41091,Sugar,Maroon 5,3ZKUZe1oUDnEUOQkzBJjf6,0.748,0.716,1,-9.388,1,0.0336,0.047200,0.000000,0.0924,0.896,120.039,234440,4,72.55700,5,1,2019-12-27
41092,The Wicked Sun,Vices I Admire,4FEYpnyVPuv5QVOps84BUc,0.503,0.709,4,-5.530,1,0.0608,0.004660,0.000206,0.0658,0.575,173.945,231360,4,22.47198,13,0,2019-12-28
41093,The Wicked Sun,Vices I Admire,03U84yM4DCDNAxrymHzK4V,0.503,0.709,4,-5.530,1,0.0608,0.004660,0.000206,0.0658,0.575,173.945,231360,4,22.47198,13,0,2019-12-28
41094,Against All Odds (Radio),Stefanie Bennett,6S25eZ2f6S0se81lxf1n8g,0.609,0.757,1,-7.171,1,0.0413,0.301000,0.000012,0.1510,0.346,120.929,184505,4,37.55677,9,0,2019-12-29


### Column Descriptions

From https://developer.spotify.com/documentation/web-api/reference/

|Value|Description|
|-|-|
|track|The name of the track.|
|artist|The person or group the track is credited to.|
|track_id|The [Spotify ID](https://developer.spotify.com/documentation/web-api/#spotify-uris-and-ids) of the track|
|danceability|How suitable a track is for dancing. A value of 0.0 is least danceable and 1.0 is most danceable.|
|energy|A perceptual measure of intensity and activity that ranges between 0 to 1. Typically, energetic tracks feel fast, loud, and noisy.|
|key|The key the track is in. Integers map to pitches using standard Pitch Class notation. E.g. 0 = C, 1 = C♯/D♭, 2 = D|
|loudness|The average loudness of a track in decibels (dB). Values typically ranges between -60 and 0 db.|
|mode|The modality (major or minor) of a track, the type of scale from which its melodic content is derived. Major is represented by 1 and minor is 0.|
|speechiness|Indicates the presence of spoken words in a track. Values above 0.66 describe tracks that are probably made entirely of spoken words. Values between 0.33 and 0.66 describe tracks that may contain both music and speech while below 0.33 most likely represent music and other non-speech-like tracks.|
|acousticness|A confidence measure indicating whether the track is acoustic. Value of 1 represents highest confidence.|
|instrumentalness|Predicts whether a track contains no vocals. The closer the value is to 1, the greater likelihood the track contains no vocal content.|
|liveness|Detects the presence of an audience in the recording. Higher liveness values represent an increased probability that the track was performed live.|
|valence|A measure to describe the musical positiveness conveyed by a track. Tracks with high valence sound more positive (e.g. happy, cheerful, euphoric), while tracks with low valence sound more negative (e.g. sad, depressed, angry).|
|tempo|The overall estimated tempo (speed or pace) of a track in beats per minute (BPM).|
|duration_ms|The duration of the track in [milliseconds](https://en.wikipedia.org/wiki/Millisecond).|
|time_signature|An estimated overall time signature of a track. The time signature is a notational convention to specify how many beats are in each bar (or measure).|
|chorus_hit|The approximate start time of the chorus, in seconds|
|sections|The number of sections in the track. Sections are defined by large variations in rhythm or timbre.|
|popularity|A value between 0 and 1, with 1 being the most popular.|
|release_date|The date the track was released.|

We can create some new columns from these, such as

* `duration_s`: duration in seconds
* `release_year`: just the year that track was released
* `link`: a link to the track on Spotify

We'll also convert the `release_date` column to only display the year. 

In [2]:
data['duration_s'] = data['duration_ms']/1000
data['release_year'] = data['release_date'].str[:4].astype(int)
data['link'] = 'https://open.spotify.com/track/' + data['track_id']
data['release_date'] = pd.to_datetime(data['release_date'], format='%Y-%m-%d')
data

,track,artist,track_id,danceability,energy,key,loudness,mode,speechiness,acousticness,...,tempo,duration_ms,time_signature,chorus_hit,sections,popularity,release_date,duration_s,release_year,link
0,Country Livin',The Lacs,1GgJ8J0Xuq3yKhirm6TIeP,0.692,0.902,5,-5.013,1,0.1590,0.155000,...,156.880,239851,4,59.70002,7,0,2018-05-04,239.851,2018,https://open.spotify.com/track/1GgJ8J0Xuq3yKhi...
1,Hopeful Shivering,Apocryphal,1nTrGqFfOubot9Vh6rt6cb,0.547,0.647,11,-9.015,0,0.0371,0.124000,...,120.045,392375,4,50.51804,17,0,2016-10-15,392.375,2016,https://open.spotify.com/track/1nTrGqFfOubot9V...
2,Punk Rawk Christmas,MxPx,1aWd5MkGpyhqENRiilEWxe,0.486,0.969,9,-4.457,1,0.0739,0.010800,...,148.067,200573,4,26.46958,10,0,2018-11-30,200.573,2018,https://open.spotify.com/track/1aWd5MkGpyhqENR...
3,I Smile,Kirk Franklin,0UQDSP8cz6WmLJckO5jqUQ,0.745,0.753,1,-2.408,1,0.1640,0.223000,...,144.057,307973,4,87.93854,9,1,2011-03-21,307.973,2011,https://open.spotify.com/track/0UQDSP8cz6WmLJc...
4,"Never Created, Never Destroyed",Jlin,6tQTZJ3lvQiJLVpKDJ47Vp,0.870,0.386,7,-8.081,1,0.4640,0.000628,...,157.845,211711,3,34.71437,10,0,2017-05-19,211.711,2017,https://open.spotify.com/track/6tQTZJ3lvQiJLVp...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41091,Sugar,Maroon 5,3ZKUZe1oUDnEUOQkzBJjf6,0.748,0.716,1,-9.388,1,0.0336,0.047200,...,120.039,234440,4,72.55700,5,1,2019-12-27,234.440,2019,https://open.spotify.com/track/3ZKUZe1oUDnEUOQ...
41092,The Wicked Sun,Vices I Admire,4FEYpnyVPuv5QVOps84BUc,0.503,0.709,4,-5.530,1,0.0608,0.004660,...,173.945,231360,4,22.47198,13,0,2019-12-28,231.360,2019,https://open.spotify.com/track/4FEYpnyVPuv5QVO...
41093,The Wicked Sun,Vices I Admire,03U84yM4DCDNAxrymHzK4V,0.503,0.709,4,-5.530,1,0.0608,0.004660,...,173.945,231360,4,22.47198,13,0,2019-12-28,231.360,2019,https://open.spotify.com/track/03U84yM4DCDNAxr...
41094,Against All Odds (Radio),Stefanie Bennett,6S25eZ2f6S0se81lxf1n8g,0.609,0.757,1,-7.171,1,0.0413,0.301000,...,120.929,184505,4,37.55677,9,0,2019-12-29,184.505,2019,https://open.spotify.com/track/6S25eZ2f6S0se81...


Now let's visualize the song lengths over the years.

In [ ]:
import plotly.express as px
px.scatter(data, x='release_date', y='duration_s', hover_data=['artist', 'track', 'link'], title='Song Duration Over Time')

We can also see if there is a relationship between `energy` and `danceability`. We'll also colour the points by `loudness` and set the visualiztion `height` to `800` so it's a little larger.

In [ ]:
px.scatter(data, x='energy', y='danceability', color='loudness', hover_data=['artist', 'track'], title='Danceability versus Energy', height=800)

If you would instead like to use a date set containing only the songs that have been played on Spotify more than a billion times, we have data from the [Billions Club Playlist](https://open.spotify.com/playlist/37i9dQZF1DX7iB3RCnBnN4).

In [ ]:
bc = pd.read_csv('https://raw.githubusercontent.com/callysto/data-files/main/data-science-and-artificial-intelligence/spotify_billions_club.csv')
bc

Then you can create columns and visualizations using the `bc` dataframe instead of using the `data` one.

In [ ]:
bc['duration_s'] = bc['duration_ms']/1000
bc['release_year'] = bc['release_date'].str[:4].astype(int)
bc = bc.rename(columns={'track_href': 'link'})
bc['release_date'] = pd.to_datetime(bc['release_date'], format='%Y-%m-%d')

px.scatter(bc, x='energy', y='danceability', color='loudness', hover_data=['artist', 'track'], title='Danceability versus Energy for Billions Club Songs')

---

<span style="color:#663399">Your **assignment** is to create at least three visualizations using Spotify data, and for each visualization write:</span>
* <span style="color:#663399">We created this visualization because</span>
* <span style="color:#663399">This visualization shows</span>
* <span style="color:#663399">Something interesting we learned from or noticed in this visualization is</span>

---

The [next notebook](07-primary-data.ipynb) will introduce you to recording and using your own (primary) data.

[![Callysto.ca License](https://github.com/callysto/curriculum-notebooks/blob/master/callysto-notebook-banner-bottom.jpg?raw=true)](https://github.com/callysto/curriculum-notebooks/blob/master/LICENSE.md)